In [44]:
import pandas as pd
import numpy as np
import re

In [47]:
df = pd.read_json(r'placeholder/bgg_GameItem.jl', lines=True)

In [48]:
df = df[df['compilation']!=1]

In [49]:
singlec_pattern = re.compile("([\S ]+)(?=:)")

In [50]:
def text_col_cleaner(col):
    
    pattern = singlec_pattern
    
    df[col] = df[col].map(lambda x: [re.findall(pattern,val)[0].strip() for val in x], na_action='ignore')
    return df

In [51]:
def cleaner_pack(frame):
    
    df = frame
    one_c = ['game_type','designer','artist','publisher','category','mechanic','family']
    df['year'] = df['year'].fillna(0).astype(int)
    df['rank'] = df['rank'].fillna(0).astype(int)

    for cat in one_c:
        text_col_cleaner(cat)

    return df

In [52]:
def encoder_columns(frame,targets):
    for t in targets:
        one_hot = pd.get_dummies(frame[t].apply(pd.Series).stack(),prefix=t).groupby(level=0).sum()
        frame = pd.concat([frame,one_hot],axis=1)
    return frame

In [53]:
def encode_column(frame,target_col,target_val):
    one_hot = pd.get_dummies(frame[target_col].apply(pd.Series).stack()).groupby(level=0).sum()
    one_hot = one_hot[target_val]
    frame = pd.concat([frame,one_hot],axis=1)

    return frame

In [54]:
df = encoder_columns(cleaner_pack(df),['mechanic','category'])

In [62]:
df = encode_column(df,'family','Crowdfunding: Kickstarter')

In [65]:
mid = df[['bgg_id','name','rank']]

In [76]:
df = pd.concat([mid, df], axis=1)

In [83]:
from pathlib import Path  

filepath = Path(r'.\BGG_Data.csv')  

filepath.parent.mkdir(parents=True, exist_ok=True)  

df.to_csv(filepath,index=False)  